# intended for making several plumed files for different models, but using same atom types

In [4]:
import pmx
import os

In [8]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/s/")

In [9]:
for a in os.walk('.'):
    files = a[2]
    
name = files[0] # for testing

In [10]:
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("S.gro") or x.endswith("R.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]
print files
#files = [files[0]] # for test

['./s17R.gro', './s17S.gro', './s7R.gro', './s7S.gro']


In [15]:
COM = []
for name in files:    
    m = pmx.Model(name)
    print name
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    nitro = [str(i.id)for i in lig.atoms if "N1" in i.name][0]



    pros = m.fetch_residues("PRO")
    for pro in pros: # get the proline with the right context
        if (m.residues[pro.id-3].resname == "THR" and
            m.residues[pro.id-2].resname == "ALA" and
            m.residues[pro.id].resname == "LYS" and
            m.residues[pro.id+1].resname == "LEU"):
            ats = pro.atoms   
    pros = m.fetch_residues("LEU")
    for pro in pros: # get the leucine with the right context
        if (m.residues[pro.id-3].resname == "LYS" and
            m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "GLU" and
            m.residues[pro.id+1].resname == "TRP"):
            ats.extend(pro.atoms)
    ids = [str(i.id) for i in ats if "H" not in i.name] # get non hydrogen atoms
    pocket = ",".join(ids)
    # COM for report
    #atoms = [m.atoms[int(z)] for z in ids]
    #COM.append([[sum(x.x[0]/len(atoms) for x in atoms)],
    #            [sum(x.x[1]/len(atoms) for x in atoms)],
    #            [sum(x.x[2]/len(atoms) for x in atoms)]])

    f = open(name[:-4]+".dat", 'w')
    f.write("\
    RESTART\n\
    p: COM ATOMS={0}\n\
    c: COM ATOMS={1}\n\
    xyz: POSITION ATOM=c NOPBC\n\
    d: DISTANCE ATOMS=c,p COMPONENTS NOPBC\n\
    d2: DISTANCE ATOMS={2},p\n\
    METAD ...\n\
    LABEL=mad\n\
    ARG=d2,d.x\n\
    PACE=250\n\
    BIASFACTOR=10\n\
    HEIGHT=1\n\
    SIGMA=0.05,0.05\n\
    FILE=HILLS{3}\n\
    GRID_MIN=0,-5\n\
    GRID_MAX=6,1.5\n\
    TEMP=300\n\
    ... METAD\n\
    UPPER_WALLS ARG=xyz.x AT=10.75 KAPPA=1000 LABEL=uwa\n\
    LOWER_WALLS ARG=xyz.x AT=8.25 KAPPA=1000 LABEL=lwa\n\
    EXTERNAL ARG=xyz.x,xyz.y,xyz.z FILE=external.pot LABEL=external\n\
    PRINT STRIDE=100 ARG=d.x,d2,mad.bias,external.bias,uwa.bias,lwa.bias,xyz.x,xyz.y,xyz.z FILE=COLVAR{3}\n\
    ".format(pocket,ligand,nitro,name[2:-4]))
    f.close()
    
    
    

./s17R.gro
./s17S.gro
./s7R.gro
./s7S.gro


In [14]:
name[2:-4]

's7S'

In [51]:
a = [[1,2,3],[2,2,2],[3,3,3]]
print sum(row[0] for row in a)

7


In [53]:
at=m.atoms[3833]

''

##### build driver files

In [43]:
for name in files:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o = [str(i.id)for i in lig.atoms if "O" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    o2 = [str(i.id)for i in lig.atoms if "O2" in i.name][0]
    c8 = [str(i.id)for i in lig.atoms if "C8" in i.name][0]


    pros = m.fetch_residues("TYR")
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            ats = pro.atoms   

    oh=[str(i.id)for i in ats if "OH" in i.name][0]
    cg=[str(i.id)for i in ats if "CG" in i.name][0]
    cb=[str(i.id)for i in ats if "CB" in i.name][0]
    ca=[str(i.id)for i in ats if "CA" in i.name][0]
    c=[str(i.id)for i in ats if "C" == i.name][0]
    name = name.split("/")[-1]
    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    a52y: DISTANCE ATOMS={p},{oh}\n\
    xi: TORSION ATOMS={oh},{cb},{ca},{c}\n\
    ang: ANGLE ATOMS={oh},{p},{o1}\n\
    ang2: ANGLE ATOMS={cg},{oh},{p}\n\
    d1: DISTANCE ATOMS={o},{oh}\n\
    d2: DISTANCE ATOMS={o2},{oh}\n\
    d3: DISTANCE ATOMS={c8},{oh}\n\
    a1: ANGLE ATOMS={oh},{p},{o}\n\
    a2: ANGLE ATOMS={oh},{p},{o2}\n\
    a3: ANGLE ATOMS={oh},{p},{c8}\n\
    PRINT STRIDE=1 ARG=a52y,xi,ang,ang2,d1,d2,d3,a1,a2,a3 FILE=driver{name}.out\n\
    ".format(p=p,oh=oh,cg=cg,cb=cb,ca=ca,c=c,o=o,o1=o1,o2=o2,c8=c8,name=name[:-4]))
    f.close()

./newABs/a49/a49S.gro
./newABs/a49/a49R.gro
./newABs/a1/a1R.gro
./newABs/a1/a1S.gro
./newABs/a7/a7S.gro
./newABs/a7/a7R.gro
./newABs/a21/a21R.gro
./newABs/a21/a21S.gro
./newABs/a46/a46R.gro
./newABs/a46/a46S.gro
./newABs/a43/a43R.gro
./newABs/a43/a43S.gro


# additional column for new fes

In [2]:
import os
import
os.chdir("/home/domain/data/kirill/AB/XOP/md/s")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith(".gro") or x.endswith(".gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./s17R.gro', './s17S.gro', './s7R.gro', './s7S.gro']


# THIS IS FOR d(Cb - P) - theta PLOTS!!!

# MAKE SURE YOU HAVE THE RIGHT CONTEXTS SELECTED FOR EVERY PROTEIN YOU DO

In [3]:
for name in files[:-2]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # s17
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "VAL" and
            m.residues[pro.id].resname == "TRP"):
            Y36L = pro.atoms   
        if (m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "ILE"):
            Y53H = pro.atoms 
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "TRP"):
            Y35H = pro.atoms  
        
   

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y36L=[str(i.id)for i in Y36L if "CB" in i.name][0]
    Y53H=[str(i.id)for i in Y53H if "CB" in i.name][0]
    Y35H=[str(i.id)for i in Y35H if "CB" in i.name][0]
    
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y36L: DISTANCE ATOMS={p},{Y36L}\n\
    Y53H: DISTANCE ATOMS={p},{Y53H}\n\
    Y35H: DISTANCE ATOMS={p},{Y35H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y36Lang: ANGLE ATOMS={Y36L},{p},{o1}\n\
    Y53Hang: ANGLE ATOMS={Y53H},{p},{o1}\n\
    Y35Hang: ANGLE ATOMS={Y35H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y33L,Y33Lang,Y36L,Y36Lang,Y53H,Y53Hang,Y35H,Y35Hang FILE=extra_CV{name}.out\n\
    ".format(p=p,Y37L=Y37L, Y33L=Y33L, Y36L=Y36L, Y53H=Y53H, Y35H=Y35H, o1=o1,name=name[:-4]))
    f.close()

./s17R.gro


NameError: name 'pmx' is not defined

In [57]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/s")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith(".gro") or x.endswith(".gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./s17R.gro', './s17S.gro', './s7R.gro', './s7S.gro']


In [5]:
for name in files[2:4]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")

        
    # s7
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "VAL" and
            m.residues[pro.id].resname == "TRP"):
            Y36L = pro.atoms   
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "MET"):
            Y35H = pro.atoms  

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y36L=[str(i.id)for i in Y36L if "CB" in i.name][0]

    Y35H=[str(i.id)for i in Y35H if "CB" in i.name][0]
    
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y36L: DISTANCE ATOMS={p},{Y36L}\n\
    Y35H: DISTANCE ATOMS={p},{Y35H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y36Lang: ANGLE ATOMS={Y36L},{p},{o1}\n\
    Y35Hang: ANGLE ATOMS={Y35H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y33L,Y33Lang,Y36L,Y36Lang,Y35H,Y35Hang FILE=extra_CV{name}.out\n\
    ".format(p=p,Y37L=Y37L, Y33L=Y33L, Y36L=Y36L, Y35H=Y35H, o1=o1,name=name[:-4]))
    f.close()

./s7R.gro
./s7S.gro


In [97]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/a5/")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("S.gro") or x.endswith("R.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./R/a5R.gro', './S/a5S.gro']


In [81]:
for name in files[:]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # a5
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            print "found Y37L"
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            print "found Y33L"
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "THR" and
            m.residues[pro.id].resname == "TYR"):
            print "found Y59H"
            Y59H = pro.atoms   
        if (m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "ILE"):
            print "found Y51H"
            Y51H = pro.atoms 
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "TRP"):
            print "found Y35H"
            Y35H = pro.atoms  
        

    Y37L=[str(i.id) for i in Y37L if "CB" in i.name][0]
    Y33L=[str(i.id) for i in Y33L if "CB" in i.name][0]
    Y59H=[str(i.id) for i in Y59H if "CB" in i.name][0]
    Y51H=[str(i.id) for i in Y51H if "CB" in i.name][0]
    Y35H=[str(i.id) for i in Y35H if "CB" in i.name][0]
    
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y59H: DISTANCE ATOMS={p},{Y59H}\n\
    Y51H: DISTANCE ATOMS={p},{Y51H}\n\
    Y35H: DISTANCE ATOMS={p},{Y35H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y59Hang: ANGLE ATOMS={Y59H},{p},{o1}\n\
    Y51Hang: ANGLE ATOMS={Y51H},{p},{o1}\n\
    Y35Hang: ANGLE ATOMS={Y35H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y33L,Y33Lang,Y59H,Y59Hang,Y51H,Y51Hang,Y35H,Y35Hang FILE=extra_CV{name}.out\n\
    ".format(p=p,Y37L=Y37L, Y33L=Y33L, Y59H=Y59H, Y51H=Y51H, Y35H=Y35H, o1=o1, name=name[:-4]))
    f.close()

./R/a5R.gro
found Y35H
found Y51H
found Y59H
found Y33L
found Y37L
./S/a5S.gro
found Y35H
found Y51H
found Y59H
found Y33L
found Y37L


In [74]:
print pro.atoms
print Y37L
i=Y37L[0]
print i

[<pmx.atom.Atom instance at 0x8919bd8>, <pmx.atom.Atom instance at 0x8919e60>, <pmx.atom.Atom instance at 0x891a128>, <pmx.atom.Atom instance at 0x891a3b0>, <pmx.atom.Atom instance at 0x891a638>, <pmx.atom.Atom instance at 0x891a8c0>, <pmx.atom.Atom instance at 0x891ab48>, <pmx.atom.Atom instance at 0x891add0>, <pmx.atom.Atom instance at 0x891b098>, <pmx.atom.Atom instance at 0x891b320>, <pmx.atom.Atom instance at 0x891b5a8>, <pmx.atom.Atom instance at 0x891b830>, <pmx.atom.Atom instance at 0x891bab8>, <pmx.atom.Atom instance at 0x891bd40>, <pmx.atom.Atom instance at 0x891bfc8>, <pmx.atom.Atom instance at 0x891c290>, <pmx.atom.Atom instance at 0x891c518>, <pmx.atom.Atom instance at 0x891c7a0>, <pmx.atom.Atom instance at 0x891ca28>, <pmx.atom.Atom instance at 0x891ccb0>, <pmx.atom.Atom instance at 0x891cf38>]
3685
3


In [83]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/a17")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("S.gro") or x.endswith("R.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./R/a17R.gro', './S/a17S.gro']


In [84]:
for name in files[:]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # a17
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "THR" and
            m.residues[pro.id].resname == "TYR"):
            Y58H = pro.atoms   
        if (m.residues[pro.id-2].resname == "ILE" and
            m.residues[pro.id].resname == "HIS"):
            Y52H = pro.atoms 
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "TRP"):
            Y33H = pro.atoms  
        if (m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "TYR"):
            Y32H = pro.atoms  

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y58H=[str(i.id)for i in Y58H if "CB" in i.name][0]
    Y52H=[str(i.id)for i in Y52H if "CB" in i.name][0]
    Y33H=[str(i.id)for i in Y33H if "CB" in i.name][0]
    Y32H=[str(i.id)for i in Y32H if "CB" in i.name][0]
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y58H: DISTANCE ATOMS={p},{Y58H}\n\
    Y52H: DISTANCE ATOMS={p},{Y52H}\n\
    Y33H: DISTANCE ATOMS={p},{Y33H}\n\
    Y32H: DISTANCE ATOMS={p},{Y32H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y58Hang: ANGLE ATOMS={Y58H},{p},{o1}\n\
    Y52Hang: ANGLE ATOMS={Y52H},{p},{o1}\n\
    Y33Hang: ANGLE ATOMS={Y33H},{p},{o1}\n\
    Y32Hang: ANGLE ATOMS={Y32H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y33L,Y33Lang,Y58H,Y58Hang,Y52H,Y52Hang,Y33H,Y33Hang,Y32H,Y32Hang FILE=extra_CV{name}.out\n\
    ".format(p=p,Y37L=Y37L, Y33L=Y33L, Y58H=Y59H, Y52H=Y52H, Y33H=Y33H, Y32H=Y32H, o1=o1,name=name[:-4]))
    f.close()

./R/a17R.gro
./S/a17S.gro


In [87]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/new/newABs/a1")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("a1R.gro") or x.endswith("a1S.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./a1R.gro', './a1S.gro']


In [91]:
for name in files[:]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # a1
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "VAL" and
            m.residues[pro.id].resname == "TRP"):
            Y35L = pro.atoms 
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "ILE"):
            Y52H = pro.atoms 
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "TRP"):
            Y35H = pro.atoms  
        if (m.residues[pro.id-2].resname == "ILE" and
            m.residues[pro.id].resname == "TYR"):
            Y54H = pro.atoms        

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y35L=[str(i.id)for i in Y35L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y54H=[str(i.id)for i in Y54H if "CB" in i.name][0]
    Y52H=[str(i.id)for i in Y52H if "CB" in i.name][0]
    Y35H=[str(i.id)for i in Y35H if "CB" in i.name][0]
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y35L: DISTANCE ATOMS={p},{Y35L}\n\
    Y54H: DISTANCE ATOMS={p},{Y54H}\n\
    Y52H: DISTANCE ATOMS={p},{Y52H}\n\
    Y35H: DISTANCE ATOMS={p},{Y35H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y35Lang: ANGLE ATOMS={Y35L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y54Hang: ANGLE ATOMS={Y54H},{p},{o1}\n\
    Y52Hang: ANGLE ATOMS={Y52H},{p},{o1}\n\
    Y35Hang: ANGLE ATOMS={Y35H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y35L,Y35Lang,Y33L,Y33Lang,Y52H,Y52Hang,Y35H,Y35Hang,Y54H,Y54Hang, FILE=extra_CV{name}.out\n\
    ".format(p=p,Y37L=Y37L, Y33L=Y33L, Y35L=Y35L, Y52H=Y52H, Y35H=Y35H,Y54H=Y54H, o1=o1,name=name[:-4]))
    f.close()

./a1R.gro
./a1S.gro


In [92]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/new/newABs/a21")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("a21R.gro") or x.endswith("a21S.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./a21R.gro', './a21S.gro']


In [93]:
for name in files[:]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # a21
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "VAL" and
            m.residues[pro.id].resname == "TRP"):
            Y35L = pro.atoms 
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  

        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "MET"):
            Y33H = pro.atoms  
      

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y35L=[str(i.id)for i in Y35L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y33H=[str(i.id)for i in Y33H if "CB" in i.name][0]
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y35L: DISTANCE ATOMS={p},{Y35L}\n\
    Y33H: DISTANCE ATOMS={p},{Y33H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y35Lang: ANGLE ATOMS={Y35L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y33Hang: ANGLE ATOMS={Y33H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y35L,Y35Lang,Y33L,Y33Lang,Y33H,Y33Hang, FILE=extra_CV{name}.out\n\
    ".format(p=p, Y37L=Y37L, Y33L=Y33L, Y35L=Y35L, Y33H=Y33H, o1=o1,name=name[:-4]))
    f.close()

./a21R.gro
./a21S.gro


In [94]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/new/newABs/a7")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("a7R.gro") or x.endswith("a7S.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./a7S.gro', './a7R.gro']


In [95]:
for name in files[:]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # a7
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "VAL" and
            m.residues[pro.id].resname == "TRP"):
            Y35L = pro.atoms 
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "ILE"):
            Y52H = pro.atoms 
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "TRP"):
            Y35H = pro.atoms  
        if (m.residues[pro.id-2].resname == "ILE" and
            m.residues[pro.id].resname == "TYR"):
            Y54H = pro.atoms        

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y35L=[str(i.id)for i in Y35L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y54H=[str(i.id)for i in Y54H if "CB" in i.name][0]
    Y52H=[str(i.id)for i in Y52H if "CB" in i.name][0]
    Y35H=[str(i.id)for i in Y35H if "CB" in i.name][0]
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y35L: DISTANCE ATOMS={p},{Y35L}\n\
    Y54H: DISTANCE ATOMS={p},{Y54H}\n\
    Y52H: DISTANCE ATOMS={p},{Y52H}\n\
    Y35H: DISTANCE ATOMS={p},{Y35H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y35Lang: ANGLE ATOMS={Y35L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y54Hang: ANGLE ATOMS={Y54H},{p},{o1}\n\
    Y52Hang: ANGLE ATOMS={Y52H},{p},{o1}\n\
    Y35Hang: ANGLE ATOMS={Y35H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y35L,Y35Lang,Y33L,Y33Lang,Y52H,Y52Hang,Y35H,Y35Hang,Y54H,Y54Hang, FILE=extra_CV{name}.out\n\
    ".format(p=p,Y37L=Y37L, Y33L=Y33L, Y35L=Y35L, Y52H=Y52H, Y35H=Y35H,Y54H=Y54H, o1=o1,name=name[:-4]))
    f.close()

./a7S.gro
./a7R.gro


In [98]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/new/newABs/a43")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("a43R.gro") or x.endswith("a43S.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./a43R.gro', './a43S.gro']


In [99]:
for name in files[:]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # a43
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "VAL" and
            m.residues[pro.id].resname == "TRP"):
            Y35L = pro.atoms 
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "ILE"):
            Y52H = pro.atoms 
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "TRP"):
            Y35H = pro.atoms  
        if (m.residues[pro.id-2].resname == "ILE" and
            m.residues[pro.id].resname == "TYR"):
            Y54H = pro.atoms        

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y35L=[str(i.id)for i in Y35L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y54H=[str(i.id)for i in Y54H if "CB" in i.name][0]
    Y52H=[str(i.id)for i in Y52H if "CB" in i.name][0]
    Y35H=[str(i.id)for i in Y35H if "CB" in i.name][0]
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y35L: DISTANCE ATOMS={p},{Y35L}\n\
    Y54H: DISTANCE ATOMS={p},{Y54H}\n\
    Y52H: DISTANCE ATOMS={p},{Y52H}\n\
    Y35H: DISTANCE ATOMS={p},{Y35H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y35Lang: ANGLE ATOMS={Y35L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y54Hang: ANGLE ATOMS={Y54H},{p},{o1}\n\
    Y52Hang: ANGLE ATOMS={Y52H},{p},{o1}\n\
    Y35Hang: ANGLE ATOMS={Y35H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y35L,Y35Lang,Y33L,Y33Lang,Y52H,Y52Hang,Y35H,Y35Hang,Y54H,Y54Hang, FILE=extra_CV{name}.out\n\
    ".format(p=p,Y37L=Y37L, Y33L=Y33L, Y35L=Y35L, Y52H=Y52H, Y35H=Y35H,Y54H=Y54H, o1=o1,name=name[:-4]))
    f.close()

./a43R.gro
./a43S.gro


In [100]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/new/newABs/a46")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("a46R.gro") or x.endswith("a46S.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./a46R.gro', './a46S.gro']


In [102]:
for name in files[:]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # a46
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "VAL" and
            m.residues[pro.id].resname == "TRP"):
            Y35L = pro.atoms 
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "TRP"):
            Y35H = pro.atoms  
        if (m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "ILE"):
            Y52H = pro.atoms        

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y35L=[str(i.id)for i in Y35L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y52H=[str(i.id)for i in Y52H if "CB" in i.name][0]
    Y35H=[str(i.id)for i in Y35H if "CB" in i.name][0]
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y35L: DISTANCE ATOMS={p},{Y35L}\n\
    Y52H: DISTANCE ATOMS={p},{Y52H}\n\
    Y35H: DISTANCE ATOMS={p},{Y35H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y35Lang: ANGLE ATOMS={Y35L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y52Hang: ANGLE ATOMS={Y52H},{p},{o1}\n\
    Y35Hang: ANGLE ATOMS={Y35H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y35L,Y35Lang,Y33L,Y33Lang,Y52H,Y52Hang,Y35H,Y35Hang, FILE=extra_CV{name}.out\n\
    ".format(p=p, Y37L=Y37L, Y33L=Y33L, Y35L=Y35L, Y52H=Y52H, Y35H=Y35H, o1=o1,name=name[:-4]))
    f.close()

./a46R.gro
./a46S.gro


In [103]:
os.chdir("/home/domain/data/kirill/AB/XOP/md/new/newABs/a49")
files = []
for a in os.walk('.'):
    fz = a[2]
    files.append([a[0]+"/"+x for x in fz if x.endswith("a49R.gro") or x.endswith("a49S.gro")])
files = [k for z in [f for f in files if len(f) > 0] for k in z]

#files = [files[0]] # for test
#files = [files[2]] # for test
print files

['./a49S.gro', './a49R.gro']


In [106]:
for name in files[:]:    
    print name
    m = pmx.Model(name)
    lig = m.fetch_residues("XOP")[0]
    ids = [str(i.id) for i in lig.atoms if "H" not in i.name] # get non hydrogen atoms
    ligand = ",".join(ids)

    p = [str(i.id)for i in lig.atoms if "P" in i.name][0]
    o1 = [str(i.id)for i in lig.atoms if "O1" in i.name][0]
    

    pros = m.fetch_residues("TYR")
    
    # a49
    for pro in pros: # get the tyrosine with the right context
        if (m.residues[pro.id-2].resname == "TRP" and
            m.residues[pro.id].resname == "GLN"):
            Y37L = pro.atoms   
        if (m.residues[pro.id-2].resname == "VAL" and
            m.residues[pro.id].resname == "TRP"):
            Y35L = pro.atoms 
        if (m.residues[pro.id-2].resname == "ASN" and
            m.residues[pro.id].resname == "VAL"):
            Y33L = pro.atoms  
        if (m.residues[pro.id-2].resname == "GLY" and
            m.residues[pro.id].resname == "ILE"):
            Y50H = pro.atoms 
        if (m.residues[pro.id-2].resname == "TYR" and
            m.residues[pro.id].resname == "TRP"):
            Y33H = pro.atoms  
        if (m.residues[pro.id-2].resname == "ILE" and
            m.residues[pro.id].resname == "TYR"):
            Y52H = pro.atoms        

    Y37L=[str(i.id)for i in Y37L if "CB" in i.name][0]
    Y35L=[str(i.id)for i in Y35L if "CB" in i.name][0]
    Y33L=[str(i.id)for i in Y33L if "CB" in i.name][0]
    Y50H=[str(i.id)for i in Y50H if "CB" in i.name][0]
    Y52H=[str(i.id)for i in Y52H if "CB" in i.name][0]
    Y33H=[str(i.id)for i in Y33H if "CB" in i.name][0]
    
    name = name.split("/")[-1]

    f = open("driver"+name[:-4]+".dat", 'w')
    f.write("\
    Y37L: DISTANCE ATOMS={p},{Y37L}\n\
    Y33L: DISTANCE ATOMS={p},{Y33L}\n\
    Y35L: DISTANCE ATOMS={p},{Y35L}\n\
    Y50H: DISTANCE ATOMS={p},{Y50H}\n\
    Y52H: DISTANCE ATOMS={p},{Y52H}\n\
    Y33H: DISTANCE ATOMS={p},{Y33H}\n\
    Y37Lang: ANGLE ATOMS={Y37L},{p},{o1}\n\
    Y33Lang: ANGLE ATOMS={Y33L},{p},{o1}\n\
    Y35Lang: ANGLE ATOMS={Y35L},{p},{o1}\n\
    Y50Hang: ANGLE ATOMS={Y50H},{p},{o1}\n\
    Y52Hang: ANGLE ATOMS={Y52H},{p},{o1}\n\
    Y33Hang: ANGLE ATOMS={Y33H},{p},{o1}\n\
    PRINT STRIDE=1 ARG=Y37L,Y37Lang,Y35L,Y35Lang,Y33L,Y33Lang,Y52H,Y52Hang,Y33H,Y33Hang,Y50H,Y50Hang, FILE=extra_CV{name}.out\n\
    ".format(p=p,Y37L=Y37L, Y33L=Y33L, Y35L=Y35L, Y52H=Y52H, Y33H=Y33H,Y50H=Y50H, o1=o1,name=name[:-4]))
    f.close()

./a49S.gro
./a49R.gro
